In [ ]:
# Create bottlenecks
%run lz_producebottlenecks.py

In [ ]:
#%%writefile lz_classifier.py
'''
revised the method for split training and testing dataset

'''
import tensorflow as tf
import sys
import math
import os
import numpy as np
import json
import argparse
import re
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.python.platform import gfile
from progress.bar import Bar

def lz_compute_bottleneck(bottleneck_dir):
    '''
    Given "bottleneck_dir", load computed bottleneck data and labels
    Output: the bottleneck data and labels
    '''
    ### LOAD DATA FROM BOTTLENECKS
    train_inputs = []
    train_labels = []
    test_inputs = []
    test_labels = []

    bottleneck_list = []
    file_glob = os.path.join(bottleneck_dir, '*.txt')
    bottleneck_list.extend(gfile.Glob(file_glob))
    # shuffle(bottleneck_list, random_state = 1)
    
    last_win_train = lz_locate_last_train_window(bottleneck_dir)

    for bottleneck_file in bottleneck_list:

        bottleneck = open(bottleneck_file)
        bottleneck_string = bottleneck.read()
        bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
        
        regex = re.compile(r'\d+')
        labels = regex.findall(bottleneck_file)
        SBP_label = int(labels[-2])
        DBP_label = int(labels[-1])
        win_index = int(labels[-3])  #extract window index for this bottleneck file
        
        if win_index > last_win_train:
            train_inputs.append(bottleneck_values)
            train_labels.append([int(SBP_label), int(DBP_label)])
        else:
            test_inputs.append(bottleneck_values)
            test_labels.append([int(SBP_label), int(DBP_label)])
            
    val_inputs = test_inputs; val_labels = test_labels
        
    return train_inputs, val_inputs, test_inputs, train_labels, val_labels, test_labels

def lz_locate_last_train_window(bottleneck_dir):
    '''
    Given window length, assign data into training, validate and testing sets
    In this analysis, validate data equivalent to testing data
    For each trial, window index belongs to first 10 second is assigned to training data
    The last 2 second is assinged to testing data.
    Output: the window index which last to the end of the first 10 second of the trial
    '''
    
    '''Extract the window length'''
    win_len = int(bottleneck_dir[-14])
    new_srate = 100
    win_step = 0.1 * new_srate
    win_len = int(bottleneck_dir[-14]) * new_srate
    
    '''Find the window index which with end at 10 second'''
    last_win_train = math.ceil((10*new_srate - win_len)/win_step + 1)
    first_win_test = 
        
    return last_win_train

def main(learning_rate = 0.0005, batch_size = 64, epochs = 2500, log_batch_step = 50):
    train_inputs, valtest_inputs, test_inputs, train_labels, valtest_labels, test_labels = lz_compute_bottleneck(bottleneck_dir)
    
    # useful info
    n_features = np.size(train_inputs, 1)
    n_labels = np.size(train_labels, 1)

    tf.reset_default_graph()
    graph = tf.get_default_graph()
    
    # Placeholders for input features and labels
    inputs = tf.placeholder(tf.float32, (None, n_features), name='inputs')
    labels = tf.placeholder(tf.float32, (None, n_labels), name='labels')
    label_SBP, label_DBP = tf.split(labels, 2, 1)

    # Setting up weights and bias
    weights = tf.Variable(tf.truncated_normal((n_features, n_labels), stddev=0.1), name='weights')
    bias = tf.Variable(tf.zeros(n_labels), name='bias')
    tf.summary.histogram('weightshist', weights)
    tf.summary.histogram('biashist', bias)
    
    # Setting up operation in fully connected layer
    predictions = tf.add(tf.matmul(inputs, weights), bias)
    pred_SBP, pred_DBP = tf.split(predictions, 2, 1)

    # Defining loss of network
    mean_squared_error_SBP = tf.losses.mean_squared_error(pred_SBP, label_SBP)
    mean_squared_error_DBP = tf.losses.mean_squared_error(pred_DBP, label_DBP)
    total_loss = mean_squared_error_SBP + mean_squared_error_DBP

    tf.summary.scalar('loss', total_loss)
    
    # Setting optimiser
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

    # For saving checkpoint after training
    saver = tf.train.Saver()

    merged = tf.summary.merge_all()
    
    # use in command line: tensorboard --logdir=path/to/log  --> to view tensorboard

    # Run tensorflow session
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        train_writer = tf.summary.FileWriter('log', sess.graph)
        tf.train.write_graph(sess.graph_def, '', 'savedgraph.pbtxt', as_text=False)

        # Running the training in batches 
        batch_count = int(math.ceil(len(train_inputs)/batch_size))

        for epoch_i in range(epochs):
            batches_pbar = tqdm(range(batch_count), desc='Epoch {:>2}/{}'.format(epoch_i+1, epochs), unit='batches')
            # The training cycle
            for batch_i in batches_pbar:
                # Get a batch of training features and labels
                batch_start = batch_i*batch_size
                batch_inputs = train_inputs[batch_start:batch_start + batch_size]
                batch_labels = train_labels[batch_start:batch_start + batch_size]
                # Run optimizer
                _, summary = sess.run([optimizer, merged], feed_dict={inputs: batch_inputs, labels: batch_labels})
                train_writer.add_summary(summary, batch_i)

            # Check mean sqaure error against validation data
            val_MSR_SBP,val_MSR_DBP,val_total_loss = sess.run([mean_squared_error_SBP,mean_squared_error_DBP,total_loss], feed_dict={inputs: val_inputs, labels: val_labels})
            print("After epoch {}, MSR_SBP: {}, MSR_DBP: {}, Total Loss: {}.".format(epoch_i+1, val_MSR_SBP,val_MSR_DBP,val_total_loss))

        test_MSR_SBP,test_MSR_DBP,test_total_loss = sess.run([mean_squared_error_SBP,mean_squared_error_DBP,total_loss], feed_dict={inputs: test_inputs, labels: test_labels})
        print ("Test MSR_SBP: {}, Test MSR_DBP: {}, Test Total Loss: {}.".format(test_MSR_SBP,test_MSR_DBP,test_total_loss))

        test_predictions = sess.run([predictions], feed_dict={inputs: test_inputs, labels: test_labels})

        g = tf.get_default_graph()
        saver.save(sess, 'savedgraph')
        
        return test_predictions

if __name__ == '__main__':
    # %run lz_producebottlenecks.py
    slim = tf.contrib.slim
    bottleneck_dir = 'VG_PPG_file2_7s_bottlenecks'
    # Setting hyperparameters
    learning_rate = 0.0005
    batch_size = 64
    epochs = 2500
    log_batch_step = 50

    main(learning_rate, batch_size, epochs, log_batch_step)


In [ ]:
import matplotlib.pyplot as plt
# from scipy.stats import ttest_ind

pred = test_predictions[0]
label = (np.array(test_labels))

DBP_pred = pred[:,1]
DBP_label = label[:,1]
DBP_R = np.corrcoef(DBP_pred, DBP_label)
DBP_MAE = (np.absolute((DBP_pred - DBP_label))).mean()
DBP_MAE_std = np.std((DBP_pred - DBP_label))
DBP_RMSE = np.sqrt(((DBP_pred - DBP_label) ** 2).mean()) 
# t_DBP, p_DBP = ttest_ind(DBP_pred, DBP_label)
print("R of DBP    = {}".format(DBP_R[0][1]))
# print("p of DBP    = {}".format(p_DBP))
print("MAE of DBP  = {} +/- {}".format(DBP_MAE, DBP_MAE_std))
print("RMSE of DBP = {}\n".format(DBP_RMSE))
plt.plot(DBP_label, DBP_pred, "o")
axes = plt.gca()
axes.set_xlim([60,120])
axes.set_ylim([60,120])
plt.show()

SBP_pred = pred[:,0]
SBP_label = label[:,0]
SBP_R = np.corrcoef(SBP_pred, SBP_label)
SBP_MAE = (np.absolute((SBP_pred - SBP_label))).mean()
SBP_MAE_std = np.std((SBP_pred - SBP_label))
SBP_RMSE = np.sqrt(((SBP_pred - SBP_label) ** 2).mean())
print("R of SBP    = {}".format(SBP_R[0][1]))
print("MAE of DBP  = {} +/- {}".format(SBP_MAE, SBP_MAE_std))
print("RMSE of SBP = {}".format(SBP_RMSE))
plt.plot(SBP_label, SBP_pred, "o")
axes = plt.gca()
axes.set_xlim([90,170])
axes.set_ylim([90,170])
plt.show()

## Scatter plot of the results across DBP and SBP

In [ ]:
comb_pred  = np.concatenate((DBP_pred, SBP_pred))
comb_label = np.concatenate((DBP_label, SBP_label))
plt.plot(comb_label, comb_pred, "bo", markersize=1)
plt.plot(comb_label, comb_label, "ro", markersize=4)
plt.xlabel('Blood Cuff Measure (mmHg)', fontsize = '16')
plt.ylabel('PPG Based Estimation (mmHg)', fontsize = '16')
plt.savefig("BP_fitting_7s.svg", format="svg", bbox_inches='tight')
axes = plt.gca()
axes.set_xlim([60,170])
axes.set_ylim([60,170])

plt.show()


## Bland-Altman analysis across DBP and SBP

In [ ]:
# plt.subplot(131)
# plt.plot(comb_label, diff_pred_label, "bo", markersize=1)
# axes = plt.gca()
# axes.set_xlim([60,170])
# axes.set_ylim([-60,60])
# plt.title("")


diff_pred_label_DBP = DBP_pred - DBP_label
diff_pred_label_SBP = SBP_pred - SBP_label
plt.subplot(131)
plt.hist(diff_pred_label_DBP, 50, normed=1, histtype='stepfilled')  # arguments are passed to np.histogram
plt.title("DBP Error")
plt.subplot(132)
plt.hist(diff_pred_label_SBP, 50, normed=1, histtype='stepfilled')  # arguments are passed to np.histogram
plt.title("SBP Error")
plt.savefig("BP_fitting_7s_error.svg", format="svg", bbox_inches='tight')
plt.show()

In [ ]:
total_DBP = diff_pred_label_DBP.size
small_than_05_DBP = sum(i < 5 for i in diff_pred_label_DBP)
small_than_10_DBP = sum(i < 10 for i in diff_pred_label_DBP)
small_than_15_DBP = sum(i < 15 for i in diff_pred_label_DBP)
percentage_small_than_05_DBP = small_than_05_DBP / total_DBP
percentage_small_than_10_DBP = small_than_10_DBP / total_DBP
percentage_small_than_15_DBP = small_than_15_DBP / total_DBP

total_SBP = diff_pred_label_SBP.size
small_than_05_SBP = sum(i < 5 for i in diff_pred_label_SBP)
small_than_10_SBP = sum(i < 10 for i in diff_pred_label_SBP)
small_than_15_SBP = sum(i < 15 for i in diff_pred_label_SBP)
percentage_small_than_05_SBP = small_than_05_SBP / total_SBP
percentage_small_than_10_SBP = small_than_10_SBP / total_SBP
percentage_small_than_15_SBP = small_than_15_SBP / total_SBP

print ("DBP < 5 = {:.2f}%, < 10 = {:.2f}%, < 15 = {:.2f}%.\n".format(percentage_small_than_05_DBP*100, percentage_small_than_10_DBP*100, percentage_small_than_15_DBP*100))
print ("SBP < 5 = {:.2f}%, < 10 = {:.2f}%, < 15 = {:.2f}%.\n".format(percentage_small_than_05_SBP*100, percentage_small_than_10_SBP*100, percentage_small_than_15_SBP*100))

In [ ]:
import lz_BlandAltman_correlation as BA
diff_pred_label = comb_pred - comb_label
# plt.subplot(131)
# plt.plot(comb_label, diff_pred_label, "bo", markersize=1)
# axes = plt.gca()
# axes.set_xlim([60,170])
# axes.set_ylim([-60,60])
# plt.title("")

# plt.subplot(132)
# plt.hist(diff_pred_label, bins='auto')  # arguments are passed to np.histogram
# plt.title("Histogram of Difference")

# plt.subplot(133)
BA.lz_BlandAltmanPlot(comb_pred, comb_label)
plt.title("Bland-Altman")
axes = plt.gca()
axes.set_xlim([60,170])

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 24,
        }

Difference = comb_pred-comb_label
mean_diff = np.mean(Difference)
Std_diff = np.std(Difference)
upper_limit = mean_diff + 1.96*Std_diff
lower_limit = mean_diff - 1.96*Std_diff
plt.text(165, mean_diff, 'Mean Diff = %0.6s' % mean_diff, fontdict=font)
plt.text(165, upper_limit, '+1.96SD = %0.6s' % upper_limit, fontdict=font)
plt.text(165, lower_limit, '-1.96SD = %0.6s' % lower_limit, fontdict=font)

plt.show()

In [ ]:
%run lz_BlandAltman_correlation

## RMSE for individual BP points

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
mean_err_uniq_est = []
std_err_uniq_est  = []
rmse_uniq_est     = []
uniq_label_list = np.unique(comb_label)
for uniq_label in uniq_label_list:
    mean_err_uniq_est.append(np.mean(Difference[comb_label == uniq_label]))
    std_err_uniq_est.append(np.std(Difference[comb_label == uniq_label]))
    rmse_uniq_est.append(sqrt(mean_squared_error(comb_pred[comb_label == uniq_label], comb_label[comb_label == uniq_label])))

plt.plot(uniq_label_list, rmse_uniq_est, 'o')
plt.xlabel('Blood Cuff Measure (mmHg)', fontsize = '16')
plt.ylabel('RMSE(mmHg)', fontsize = '16')
plt.show()
np.mean(rmse_uniq_est)